<a href="https://colab.research.google.com/github/PabloParadaSouto/Automatica/blob/master/DDPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gym

# Definir la red neuronal actor
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, action_dim)
        self.max_action = max_action

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x)) * self.max_action
        return x

# Definir la red neuronal crítica
class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim + action_dim, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, 1)

    def forward(self, x, u):
        x = F.relu(self.fc1(torch.cat([x, u], 1)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Definir el algoritmo DDPG
class DDPG:
    def __init__(self, state_dim, action_dim, max_action):
        self.actor = Actor(state_dim, action_dim, max_action).cuda()
        self.actor_target = Actor(state_dim, action_dim, max_action).cuda()
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-4)

        self.critic = Critic(state_dim, action_dim).cuda()
        self.critic_target = Critic(state_dim, action_dim).cuda()
        self.critic_target.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=1e-3)

        self.max_action = max_action

    #Selecciona la accion y la devuelve aplanada (flatten())
    def select_action(self, state):
        state = torch.Tensor(state.reshape(1, -1)).cuda()
        return self.actor(state).cpu().data.numpy().flatten()

    #.cuda() sireve para usar targeta grafica y que el proceso sea mas rapido
    def train(self, replay_buffer, batch_size=64, gamma=0.99, tau=0.005):
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        state = torch.Tensor(state).cuda()
        #selecciona la accion segun el estado actual
        action = torch.Tensor(action).cuda()
        next_state = torch.Tensor(next_state).cuda()
        reward = torch.Tensor(reward).reshape((batch_size, 1)).cuda()
        not_done = torch.Tensor(not_done).reshape((batch_size, 1)).cuda()

        #selecciona la accion siguiente segun el estado siguiente
        next_action = self.actor_target(next_state)
        #obtiene el resultado de la red target del critic
        target_Q = self.critic_target(next_state, next_action)
        #ecuacion de bellman, el detach() es para que no acumule gradientes
        target_Q = reward + (not_done * gamma * target_Q).detach()

        #q valor predicho por mi red critic
        current_Q = self.critic(state, action)

        #error entre la red critic y la target
        critic_loss = F.mse_loss(current_Q, target_Q)

        #reinicia los gradientes, backpropagation y aplica el optimizador
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        #error de la red actor
        actor_loss = -self.critic(state, self.actor(state)).mean()

        #reinicia los gradientes, backpropagation y aplica el optimizador
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        #obteine los parametros de la red critic y de la target
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

        #obteine los parametros de la red actor y de la target
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

# Configuración de la semilla aleatoria
torch.manual_seed(0)
np.random.seed(0)

# Definir el entorno
env = gym.make('Pendulum-v0')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

# Inicializar el agente DDPG
agent = DDPG(state_dim, action_dim, max_action)

# Configuración de hiperparámetros
max_episodes = 1000
max_steps = 500
batch_size = 64

# Entrenamiento
for episode in range(max_episodes):
    state = env.reset()
    episode_reward = 0

    for step in range(max_steps):
        action = agent.select_action(state)
        next_state, reward, done, _ = env.step(action)
        agent.train(replay_buffer)
        state = next_state
        episode_reward += reward

        if done:
            break

    print("Episode: {}, Reward: {}".format(episode, episode_reward))
